# Chapter 3: Beginning Data Analysis

## Recipes
* [Developing a data analysis routine](#Developing-a-data-analysis-routine)
* [Reducing memory by changing data types](#Reducing-memory-by-changing-data-types)
* [Selecting the smallest of the largest](#Selecting-the-smallest-of-the-largest)
* [Selecting the largest of each group by sorting](#Selecting-the-largest-of-each-group-by-sorting)
* [Replicating nlargest with sort_values](#Replicating-nlargest-with-sort_values)

In [1]:
import pandas as pd
import numpy as np
from IPython.display import display
pd.options.display.max_columns = 100

This writes the csv file for the movie data

In [2]:
#these import all the required libraries
from bs4 import BeautifulSoup
import json
import requests
from urllib.request import urlopen 

def main():
    #this reads imdb url using BeautifulSoup
    response = requests.get("https://www.imdb.com/list/ls003073623/")
    soup = BeautifulSoup(response.content,'html.parser')

    #this reads the type attribute from the source and load them into Json object
    x = soup.find(type='application/ld+json').text
    d = json.loads(x)

    #this gets all the objects in the itemListElement
    get_itemlist = d['about']['itemListElement']

    #this slices the movie ids from the string and put them into a list
    movie_ids = []
    for i in get_itemlist:
        movie_ids.append(i['url'][7:16])

    import csv
    movie_data = open('HeyMovieData.csv', 'w')
    csvwriter = csv.writer(movie_data)

    headingsWritten = False
    for item in movie_ids:
        #this runs the url using movie ids and find the API
        with urlopen("http://www.omdbapi.com/?i="+item+"&apikey=a83b77e8") as url:
            data = json.loads(url.read().decode()) 

            #these deal with the missing data in BoxOffice as well as miss-spelling of values  
            if 'BoxOffice' in data:
                if data['BoxOffice'] == '&pound;737,565':
                    data['BoxOffice'] = '737,565'
                    data['BoxOffice'] = float("".join( data['BoxOffice'].split(",")))
                else:
                    data['BoxOffice'] = data['BoxOffice'].replace('N/A','$0')
                    data['BoxOffice'] = data['BoxOffice'][1:]
                    data['BoxOffice'] = float("".join( data['BoxOffice'].split(",")))
            else: 
                data['BoxOffice'] = 0

            #these create columns if they haven't already existed and give them 'N/A/ value 
            if 'DVD' not in data:
                data['DVD'] = 'N/A'

            if 'Production' not in data:
                data['Production'] = 'N/A'

            if 'Website' not in data:
                data['Website'] = 'N/A'

            #this extracts the year for each movie, and slice them into 4 digits
            movie_year = data['Year'][0:4]
            data['Year'] = movie_year

            #this replaces the 'N/A' values in imdbVotes to 0
            data['imdbVotes']= data['imdbVotes'].replace('N/A','0')

            #this runs the url for each year of the movie and bring back their inflation adjusted price   
            url = "https://data.bls.gov/cgi-bin/cpicalc.pl?cost1=1&year1="+movie_year+"01&year2=201808"
            response = requests.get(url)
            soup = BeautifulSoup(response.content,'html.parser')
            price = soup.find(id='answer').text

            #this removes the dollar sign and turns the string into float
            price = float( price[1:])

            #this calculates the adjusted boxoffice price
            Adjusted_BoxOffice = data['BoxOffice'] * price

            #this converts adjusted office value from float to string
            w = json.dumps(Adjusted_BoxOffice)

            #this adds a new key and value to json data object
            data['adjusted_boxOffice'] = w

            #this deletes the Ratings and TotalSeasons columns (if exist)
            del data['Ratings']

            if 'totalSeasons' in data:
                del data['totalSeasons']

            #this adds a field type to json data objects
            field_type = {'LineItems': [data]}
            q = json.dumps(field_type)
            movie_parsed = json.loads(q)
            mov_data = movie_parsed['LineItems']

            #these write the heading as well as the values to each row of the csv file
            if not headingsWritten:
                header = mov_data[0].keys()
                csvwriter.writerow(header)
                headingsWritten = True
            csvwriter.writerow(mov_data[0].values())
    movie_data.close()
    

# Developing a data analysis routine

This loads the HeyMovieData.csv and prints the first five rows of the table

In [3]:
movie = pd.read_csv('http://knuth.luther.edu/~doroel01/HeyMovieData.csv')

In [4]:
movie.head()

,adjusted_boxOffice,Rated,imdbRating,Writer,Awards,Poster,Production,imdbID,Website,Year,Response,DVD,Actors,Director,Title,Type,Runtime,Metascore,Genre,Language,Plot,BoxOffice,Country,imdbVotes,Released
0,0.0,R,9.2,"Mario Puzo (screenplay by), Francis Ford Coppo...",Won 3 Oscars. Another 24 wins & 28 nominations.,https://m.media-amazon.com/images/M/MV5BM2MyNj...,Paramount Pictures,tt0068646,http://www.thegodfather.com,1972,True,09 Oct 2001,"Marlon Brando, Al Pacino, James Caan, Richard ...",Francis Ford Coppola,The Godfather,movie,175 min,100.0,"Crime, Drama","English, Italian, Latin",The aging patriarch of an organized crime dyna...,0.0,USA,"1,370,696",24 Mar 1972
1,0.0,PG,8.5,"Lawrence Kasdan (screenplay by), George Lucas ...",Won 4 Oscars. Another 30 wins & 23 nominations.,https://m.media-amazon.com/images/M/MV5BMjA0OD...,Paramount Pictures,tt0082971,http://www.indianajones.com/raiders/,1981,True,21 Oct 2003,"Harrison Ford, Karen Allen, Paul Freeman, Rona...",Steven Spielberg,Raiders of the Lost Ark,movie,115 min,85.0,"Action, Adventure","English, German, Hebrew, Spanish, Arabic, Nepali","In 1936, archaeologist and adventurer Indiana ...",0.0,USA,"776,796",12 Jun 1981
2,14736070.8,PG,8.8,"Leigh Brackett (screenplay by), Lawrence Kasda...",Won 1 Oscar. Another 21 wins & 19 nominations.,https://m.media-amazon.com/images/M/MV5BYmU1ND...,Twentieth Century Fox,tt0080684,http://www.starwars.com/episode-v/,1980,True,21 Sep 2004,"Mark Hamill, Harrison Ford, Carrie Fisher, Bil...",Irvin Kershner,Star Wars: Episode V - The Empire Strikes Back,movie,124 min,82.0,"Action, Adventure, Fantasy, Sci-Fi",English,After the rebels are brutally overpowered by t...,4548170.0,USA,"1,007,473",20 Jun 1980
3,0.0,R,9.3,"Stephen King (short story ""Rita Hayworth and S...",Nominated for 7 Oscars. Another 19 wins & 32 n...,https://m.media-amazon.com/images/M/MV5BMDFkYT...,Columbia Pictures,tt0111161,NaN,1994,True,27 Jan 1998,"Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",Frank Darabont,The Shawshank Redemption,movie,142 min,80.0,Drama,English,Two imprisoned men bond over a number of years...,0.0,USA,"2,005,476",14 Oct 1994
4,0.0,PG,8.0,"Peter Benchley (screenplay), Carl Gottlieb (sc...",Won 3 Oscars. Another 11 wins & 18 nominations.,https://m.media-amazon.com/images/M/MV5BMmVmOD...,Universal Pictures,tt0073195,http://www.jaws25.com/,1975,True,11 Jul 2000,"Roy Scheider, Robert Shaw, Richard Dreyfuss, L...",Steven Spielberg,Jaws,movie,124 min,87.0,"Adventure, Drama, Thriller",English,"A local sheriff, a marine biologist and an old...",0.0,USA,"489,473",20 Jun 1975


The shape method tells us the size of our table. It shows 500 rows and 25 columns

In [5]:
movie.shape

(500, 25)

The info method lists the data type of each column, number of non-missing values, and memory usage 

In [6]:
movie.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 25 columns):
adjusted_boxOffice    500 non-null float64
Rated                 492 non-null object
imdbRating            500 non-null float64
Writer                497 non-null object
Awards                483 non-null object
Poster                500 non-null object
Production            490 non-null object
imdbID                500 non-null object
Website               189 non-null object
Year                  500 non-null int64
Response              500 non-null bool
DVD                   489 non-null object
Actors                499 non-null object
Director              499 non-null object
Title                 500 non-null object
Type                  500 non-null object
Runtime               500 non-null object
Metascore             379 non-null float64
Genre                 500 non-null object
Language              499 non-null object
Plot                  500 non-null object
BoxOffice    

This provides us with the summary statistics for the numerical columns and transpose the DataFrame for more
readable output

In [7]:
movie.describe(include=[np.number]).T

,count,mean,std,min,25%,50%,75%,max
adjusted_boxOffice,500.0,2.797046e+07,9.547197e+07,0.0,0.0,0.0,0.0,659430000.0
imdbRating,500.0,7.871800e+00,4.879148e-01,5.7,7.6,7.9,8.2,9.3
Year,500.0,1.980032e+03,2.077957e+01,1924.0,1966.0,1984.0,1998.0,2008.0
Metascore,379.0,7.920844e+01,1.240184e+01,41.0,70.0,80.0,89.0,100.0
BoxOffice,500.0,2.042119e+07,6.997932e+07,0.0,0.0,0.0,0.0,533316061.0


This provides us summary statistics for the object and categorical columns

In [8]:
movie.describe(include=[np.object, pd.Categorical]).T

,count,unique,top,freq
Rated,492,12,R,210
Writer,497,472,Woody Allen,4
Awards,483,415,1 win.,10
Poster,500,500,https://m.media-amazon.com/images/M/MV5BMTg0Nj...,1
Production,490,144,Paramount Pictures,45
imdbID,500,500,tt0058182,1
Website,189,183,http://www.indianajones.com/,3
DVD,489,329,29 Jun 1999,6
Actors,499,498,"Mark Hamill, Harrison Ford, Carrie Fisher, Bil...",2
Director,499,299,Steven Spielberg,11


## There's more...

This shows that it's possible to specify the exact quantiles returned from the describe method when used
with numeric columns

In [9]:
movie.describe(include=[np.number],
 percentiles=[.01, .05, .10, .25, .5,
 .75, .9, .95, .99]).T

,count,mean,std,min,1%,5%,10%,25%,50%,75%,90%,95%,99%,max
adjusted_boxOffice,500.0,2.797046e+07,9.547197e+07,0.0,0.000,0.00,0.0,0.0,0.0,0.0,6.399530e+07,2.134655e+08,5.290190e+08,659430000.0
imdbRating,500.0,7.871800e+00,4.879148e-01,5.7,6.399,7.10,7.3,7.6,7.9,8.2,8.400000e+00,8.600000e+00,8.900000e+00,9.3
Year,500.0,1.980032e+03,2.077957e+01,1924.0,1928.980,1940.95,1948.0,1966.0,1984.0,1998.0,2.004000e+03,2.006000e+03,2.007010e+03,2008.0
Metascore,379.0,7.920844e+01,1.240184e+01,41.0,46.780,57.90,63.0,70.0,80.0,89.0,9.500000e+01,9.800000e+01,1.000000e+02,100.0
BoxOffice,500.0,2.042119e+07,6.997932e+07,0.0,0.000,0.00,0.0,0.0,0.0,0.0,4.494039e+07,1.674593e+08,3.734494e+08,533316061.0


# Reducing memory by changing data types

## How to do it...

After reading in our movie dataset, we select a few columns of different data types that will show how much memory 
may be saved:

In [10]:
movie = pd.read_csv('http://knuth.luther.edu/~doroel01/HeyMovieData.csv')
different_cols = ['Title', 'Genre', 'BoxOffice', 'imdbVotes', 'Year']
col2 = movie.loc[:, different_cols]
col2.head()

,Title,Genre,BoxOffice,imdbVotes,Year
0,The Godfather,"Crime, Drama",0.0,"1,370,696",1972
1,Raiders of the Lost Ark,"Action, Adventure",0.0,"776,796",1981
2,Star Wars: Episode V - The Empire Strikes Back,"Action, Adventure, Fantasy, Sci-Fi",4548170.0,"1,007,473",1980
3,The Shawshank Redemption,Drama,0.0,"2,005,476",1994
4,Jaws,"Adventure, Drama, Thriller",0.0,"489,473",1975


The dtypes method inspects the data types of each column

In [11]:
col2.dtypes

Title         object
Genre         object
BoxOffice    float64
imdbVotes     object
Year           int64
dtype: object

This finds the memory usage of each column with the memory_usage method

In [12]:
original_mem = col2.memory_usage(deep=True)
original_mem

Index           80
Title        36542
Genre        37790
BoxOffice     4000
imdbVotes    31803
Year          4000
dtype: int64

To save memory, we can change object data types to categorical if they have a reasonably low number of unique
values. The below methos checks the number of unique values for both the object columns

In [13]:
col2.select_dtypes(include=['object']).nunique()

Title        498
Genre        179
imdbVotes    500
dtype: int64

The Title column is a good candidate to convert to Categorical as less than one percent of its values are unique

In [14]:
col2['Genre'] = col2['Genre'].astype('category')
col2.dtypes

Title          object
Genre        category
BoxOffice     float64
imdbVotes      object
Year            int64
dtype: object

Here, we compute the memory usage again

In [15]:
new_mem = col2.memory_usage(deep=True)
new_mem

Index           80
Title        36606
Genre        20325
BoxOffice     4000
imdbVotes    31803
Year          4000
dtype: int64

This compares the original memory usage with our updated memory usage. 
The Title column has shrunk to just 1.6 of its original size

In [16]:
new_mem / original_mem

Index        1.000000
Title        1.001751
Genre        0.537841
BoxOffice    1.000000
imdbVotes    1.000000
Year         1.000000
dtype: float64

## There's more...

To get a better idea of how object data type columns differ from integers and floats, a single
value from each one of these columns can be modified and the resulting memory usage
displayed. The Year and Genre columns are of int64 and object types, respectively:

In [17]:
movie.loc[0, 'Year'] = 10000000
movie.loc[0, 'Genre'] = movie.loc[0, 'Genre'] + 'a'
movie[['Year', 'Genre']].memory_usage(deep=True)

Index       80
Year      4000
Genre    37847
dtype: int64

Each of the following method below produces the same result

In [18]:
movie.describe(include=['int64', 'float64']).T
movie.describe(include=[np.int64, np.float64]).T
movie.describe(include=['int', 'float']).T
movie.describe(include=['number']).T

,count,mean,std,min,25%,50%,75%,max
adjusted_boxOffice,500.0,2.797046e+07,9.547197e+07,0.0,0.0,0.0,0.0,659430000.0
imdbRating,500.0,7.871800e+00,4.879148e-01,5.7,7.6,7.9,8.2,9.3
Year,500.0,2.197609e+04,4.471250e+05,1924.0,1966.0,1984.5,1998.0,10000000.0
Metascore,379.0,7.920844e+01,1.240184e+01,41.0,70.0,80.0,89.0,100.0
BoxOffice,500.0,2.042119e+07,6.997932e+07,0.0,0.0,0.0,0.0,533316061.0


These strings can be similarly used when changing types

In [19]:
movie['Year'] = movie['Year'].astype('float16')
movie['imdbRating'] = movie['imdbRating'].astype('int8')

It's possible to see the enormous memory difference between the minimal
RangeIndex and Int64Index, which stores every row index in memory

In [20]:
movie.index = pd.Int64Index(movie.index)
movie.index.memory_usage() # previously was just 80

4000

# Selecting the smallest of the largest

# How to do it...

These read in the movie dataset, and select the column Title, imdbRating, and BoxOffice

In [21]:
movie = pd.read_csv('http://knuth.luther.edu/~doroel01/HeyMovieData.csv')
movie2 = movie[['Title', 'imdbRating', 'BoxOffice']]
movie2.head()

,Title,imdbRating,BoxOffice
0,The Godfather,9.2,0.0
1,Raiders of the Lost Ark,8.5,0.0
2,Star Wars: Episode V - The Empire Strikes Back,8.8,4548170.0
3,The Shawshank Redemption,9.3,0.0
4,Jaws,8.0,0.0


The nlargest method is used to select the top 100 movies by imdbRating

In [22]:
movie2.nlargest(100,'imdbRating').head()

,Title,imdbRating,BoxOffice
3,The Shawshank Redemption,9.3,0.0
0,The Godfather,9.2,0.0
14,The Dark Knight,9.0,533316061.0
18,The Godfather: Part II,9.0,0.0
8,Pulp Fiction,8.9,0.0


The nsmallest method is used to return the five lowest BoxOffice films among those with a top 100 score

In [23]:
 movie2.nlargest(100,'imdbRating').nsmallest(5,'BoxOffice')

,Title,imdbRating,BoxOffice
3,The Shawshank Redemption,9.3,0.0
0,The Godfather,9.2,0.0
18,The Godfather: Part II,9.0,0.0
8,Pulp Fiction,8.9,0.0
24,"The Good, the Bad and the Ugly",8.9,0.0


# Selecting the largest of each group by sorting

# How to do it...

These read in the movie dataset and slim it down to just the three columns we care
about, Title, Year, and imdbRating

In [24]:
movie = pd.read_csv('http://knuth.luther.edu/~doroel01/HeyMovieData.csv')
movie2 = movie[['Title', 'Year', 'imdbRating']]

The sort_values method is used to sort the DataFrame by Year. The
default behavior sorts from the smallest to largest. Thus, we use the ascending parameter to invert this behavior by 
setting it equal to True:

In [25]:
movie2.sort_values('Year', ascending=False).head()

,Title,Year,imdbRating
452,Indiana Jones and the Kingdom of the Crystal S...,2008,6.2
14,The Dark Knight,2008,9.0
405,Iron Man,2008,7.9
372,WALL·E,2008,8.4
393,Cloverfield,2008,7.0


On the above cell, only the year was sorted. To sort multiple columns at once, we use a list. For instance,
we sort both Year and imdbRating below

In [26]:
movie3 = movie2.sort_values(['Year','imdbRating'], ascending=False)
movie3.head()

,Title,Year,imdbRating
14,The Dark Knight,2008,9.0
372,WALL·E,2008,8.4
405,Iron Man,2008,7.9
393,Cloverfield,2008,7.0
452,Indiana Jones and the Kingdom of the Crystal S...,2008,6.2


This uses the drop_duplicates method to keep only the first row of every year

In [27]:
movie_top_year = movie3.drop_duplicates(subset='Year')
movie_top_year.head()

,Title,Year,imdbRating
14,The Dark Knight,2008,9.0
143,There Will Be Blood,2007,8.1
207,The Departed,2006,8.5
80,Batman Begins,2005,8.3
72,Eternal Sunshine of the Spotless Mind,2004,8.3


# There's more...

The following sorts Year and Rated in descending order and BoxOffice in ascending order. It then finds the lowest
BoxOffice film for each year and Rated group

In [28]:
movie4 = movie[['Title', 'Year','Rated', 'BoxOffice']]
movie4_sorted = movie4.sort_values(['Year','Rated', 'BoxOffice'],ascending=[False, False, True])
movie4_sorted.drop_duplicates(subset=['Year','Rated']).head(10)

,Title,Year,Rated,BoxOffice
393,Cloverfield,2008,PG-13,79952254.0
372,WALL·E,2008,G,223749872.0
354,Sunshine,2007,R,0.0
462,Juno,2007,PG-13,143380890.0
378,Ratatouille,2007,G,112408657.0
328,The Lives of Others,2006,R,11200000.0
483,The Fountain,2006,PG-13,10046093.0
269,The Death of Mr. Lazarescu,2005,R,0.0
329,Star Wars: Episode III - Revenge of the Sith,2005,PG-13,0.0
422,Kill Bill: Vol. 2,2004,R,0.0


# Replicating nlargest with sort_values

# How to do it...

This recreates the result from the final step of the Selecting the smallest from the largest recipe

In [29]:
movie = pd.read_csv('http://knuth.luther.edu/~doroel01/HeyMovieData.csv')
movie2 = movie[['Title', 'imdbRating', 'BoxOffice']]
movie_smallest_largest = movie2.nlargest(100, 'imdbRating').nsmallest(5, 'BoxOffice')
movie_smallest_largest

,Title,imdbRating,BoxOffice
3,The Shawshank Redemption,9.3,0.0
0,The Godfather,9.2,0.0
18,The Godfather: Part II,9.0,0.0
8,Pulp Fiction,8.9,0.0
24,"The Good, the Bad and the Ugly",8.9,0.0


The sort_values method replicates the first part of the expression and grabs the first 100 rows with the head method

In [30]:
movie2.sort_values('imdbRating', ascending=False).head(100)

,Title,imdbRating,BoxOffice
3,The Shawshank Redemption,9.3,0.0
0,The Godfather,9.2,0.0
18,The Godfather: Part II,9.0,0.0
14,The Dark Knight,9.0,533316061.0
43,Schindler's List,8.9,0.0
8,Pulp Fiction,8.9,0.0
33,The Lord of the Rings: The Return of the King,8.9,364000000.0
71,12 Angry Men,8.9,0.0
24,"The Good, the Bad and the Ugly",8.9,0.0
2,Star Wars: Episode V - The Empire Strikes Back,8.8,4548170.0


Using the sort_values with head again to grab the lowest five by BoxOffice

In [31]:
movie2.sort_values('imdbRating', ascending=False).head(100).sort_values('BoxOffice').head()

,Title,imdbRating,BoxOffice
3,The Shawshank Redemption,9.3,0.0
179,To Kill a Mockingbird,8.3,0.0
94,Yojimbo,8.3,0.0
98,Toy Story,8.3,0.0
97,North by Northwest,8.3,0.0


# How it works...

This sorts the largest imdbRating from the tail or end if the table

In [32]:
movie2.nlargest(100, 'imdbRating').tail()

,Title,imdbRating,BoxOffice
458,Ikiru,8.3,0.0
465,Snatch,8.3,30093107.0
10,Raging Bull,8.2,0.0
12,Chinatown,8.2,0.0
19,Blade Runner,8.2,0.0


This sorts the imdbRating for the tail of 100 movies from largest to smallest

In [33]:
movie2.sort_values('imdbRating', ascending=False).head(100).tail()

,Title,imdbRating,BoxOffice
39,Vertigo,8.3,0.0
80,Batman Begins,8.3,204100000.0
343,The Last Waltz,8.2,0.0
168,Viridiana,8.2,0.0
59,Come and See,8.2,0.0


One interesting thing I found:
1. There's a low positive correlation between the IMDB Rating and the IMDB Votes, with R^2 = 0.25533. The high IMDB rating for a movie is somehow determined by the a high number of people voting for the particular movie and vice versa. For instance, "Shawshank Redemption" gets the highest IMDB rating (9.3) among the 500 movies correlates with the highest number of IMDB voting (2,005,476). In contrast, "Flesh" gets the lowest IMDB rating (5.7) among the 500 movies correlates with small number of IMDB voting(2,062), but not necessarily the smallest one (1,839). This is found using the scatter plot involving IMDB rating as a response variable(Y-axis) and IMDB voting as predictors (X-axis) along with the presence of the best fit line in the graph.